In [15]:
import pandas as pd
import random
import numpy as np
from tqdm import tqdm
import ipdb
import re
from collections import Counter
from sklearn.metrics import f1_score, precision_score, recall_score

import matplotlib.pyplot as plt
# import mplcursors
import seaborn as sns
%matplotlib inline
sns.set(style='darkgrid', context='notebook', rc={'figure.figsize':(14,10)}, font_scale=2)

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('chained_assignment',None)

# Set random seeds for reproducibility on a specific machine
random.seed(1)
np.random.seed(1)
np.random.RandomState(1)

RandomState(MT19937) at 0x7FBBA450FB40

In [2]:
df = pd.read_csv('predictions-multitask.tsv', sep='\t')
df.shape

(306, 31)

In [3]:
df

,TweetId,tweet,username,mentname,Date,Predom,Feeling,Behavior,Admiration,Admiration_Int,Anger,Anger_Int,Disgust,Disgust_Int,Fear,Fear_Int,Interest,Interest_Int,Joy,Joy_Int,Sadness,Sadness_Int,Surprise,Surprise_Int,group,party,Split,Specificity,emot,group_pred,emot_pred
0,1005129335160918018,"BREAKING —> The U.S. House just passed a measure I led w\/@repbrianmast to help disabled veterans transition to the civilian workforce by increasing support for the Wounded Warrior Program, which offers these vets 2-yr paid fellowships in House offices. #FlaPol #Sayfie #SSNalerts",repstephmurphy,repbrianmast,2018-06-08,predom,warm,app,False,2.000000,False,2.0,False,2.000000,False,2.0,False,2.0,False,2.000000,False,2.000000,False,2.0,0,D,dev,4.402458,[0. 0. 0. 0. 0. 0. 0. 0.],0.0,"[0, 0, 0, 0, 1, 0, 0, 0]"
1,1007345095153250304,"Today, I introduced the Safe Home Act w\/ @reptommarino to make it clear that child welfare authorities can investigate cases of “rehoming”.",jimlangevin,reptommarino,2018-06-14,predom,warm,app,False,2.000000,False,2.0,False,2.000000,False,2.0,False,2.0,False,1.666667,False,2.000000,False,2.0,0,D,dev,3.640366,[0. 0. 0. 0. 0. 0. 0. 0.],0.0,"[0, 0, 0, 0, 0, 0, 0, 0]"
2,1016423523756204032,"With far too many parents struggling with addiction, grandparents are increasingly coming to the rescue & assuming a parental role. @senbobcasey & I are delighted to announce that our bill to assist these “grandfamilies” has been signed into law.",senatorcollins,senbobcasey,2018-07-09,predom,warm,app,False,2.000000,False,2.0,False,2.000000,False,2.0,False,2.0,True,2.000000,False,2.000000,False,2.0,0,R,dev,3.960964,[0. 0. 0. 0. 0. 1. 0. 0.],0.0,"[0, 0, 0, 0, 1, 1, 0, 0]"
3,1030191720879804416,"With the 2020 Census fast approaching, I joined @SenKamalaHarris and @senatorcarper in introducing legislation to ensure the approximately 10 million Americans who identify as LGBTQ are properly counted and represented in Census data collection efforts.",senduckworth,senatorcarper,2018-08-16,predom,warm,app,False,2.000000,False,2.0,False,2.000000,False,2.0,False,2.0,False,1.666667,False,2.000000,False,2.0,1,D,dev,4.325275,[0. 0. 0. 0. 0. 0. 0. 0.],1.0,"[0, 0, 0, 0, 0, 0, 0, 0]"
4,1033514401473519616,"Rest In Peace, @senjohnmccain. Thank you for serving our country with dignity and honor, as a Navy Pilot, a POW, elected official and a presidential candidate. You were a patriot and a true American hero. My deepest condolences and prayers to his family and loved ones.",repkihuen,senjohnmccain,2018-08-25,predom,warm,app,True,3.000000,False,2.0,False,2.000000,False,2.0,False,2.0,False,2.000000,True,2.333333,False,2.0,0,D,dev,4.142287,[1. 0. 0. 0. 0. 0. 1. 0.],0.0,"[1, 0, 0, 0, 0, 0, 0, 0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,974709958280347649,Extremely sad to hear of the loss of my colleague & dean of the NY delegation @louiseslaughter. She recently brought together the NY delegation in her office. Louise was a staunch advocate for her beliefs and her legacy will forever grace the halls of Congress.,repleezeldin,louiseslaughter,2018-03-16,predom,warm,app,True,2.666667,False,2.0,False,2.000000,False,2.0,False,2.0,False,2.000000,True,2.333333,False,2.0,0,R,dev,4.460001,[1. 0. 0. 0. 0. 0. 1. 0.],1.0,"[1, 0, 0, 0, 0, 0, 0, 0]"
302,975790655950721024,"I am proud to work alongside @repmarcykaptur. I want to congratulate her for being the longest-serving women in House history, her legacy and work in Congress will pave the way for other women in politics. #WomenMakeHistory #WomensHistoryMonth",repadamsmith,repmarcykaptur,2018-03-19,predom,warm,app,True,2.666667,False,2.0,False,2.000000,False,2.0,False,2.0,True,2.333333,False,2.000000,False,2.0,1,D,dev,4.258981,[1. 0. 0. 0. 0. 1. 0. 0.],1.0,"[1, 0, 0, 0, 0, 1, 0, 0]"
303,978022747623383040,Just announced my town halls for March 27 and April 2 & 4! Looking forward to discussing gun violence and what actions Congr

In [7]:
df['group_pred'] = df['group_pred'].astype(int)
df['error'] = df.apply(lambda x: 'right' if x['group_pred']==x['group'] else 'wrong', axis=1)
wrong = df[df['group']!=df['group_pred']]
right = df[df['group']==df['group_pred']]
wrong.shape, right.shape

((63, 32), (243, 32))

Model performs well on joy and admiration, but not on disgust

In [8]:
df[(df['Disgust']==True) & (df['error']=='right')].shape, df[(df['Disgust']==True) & (df['error']=='wrong')].shape

((30, 32), (2, 32))

In [9]:
df[(df['Sadness']==True) & (df['error']=='right')].shape, df[(df['Sadness']==True) & (df['error']=='wrong')].shape

((7, 32), (4, 32))

In [10]:
df[(df['Disgust']==True) & (df['error']=='wrong')]

,TweetId,tweet,username,mentname,Date,Predom,Feeling,Behavior,Admiration,Admiration_Int,Anger,Anger_Int,Disgust,Disgust_Int,Fear,Fear_Int,Interest,Interest_Int,Joy,Joy_Int,Sadness,Sadness_Int,Surprise,Surprise_Int,group,party,Split,Specificity,emot,group_pred,emot_pred,error
140,461943113397706752,Paula is one of the #FacesoftheUnemployed & had always worked since she was 15 years old. #RenewUI @speakerboehner.,repcicilline,speakerboehner,2014-05-01,predom,cold,disapp,False,2.0,True,2.333333,True,2.333333,False,2.0,False,2.0,False,2.0,False,2.0,False,2.0,0,D,dev,3.344323,[0. 1. 1. 0. 0. 0. 0. 0.],1,"[0, 0, 0, 0, 1, 0, 0, 0]",wrong
241,803751147240439808,"Trump selected @reptomprice for HHS Secretary. Price has undeniable history of cutting access to healthcare to millions, especially women",repadamschiff,reptomprice,2016-11-29,predom,cold,disapp,False,2.0,True,2.333333,True,2.333333,False,2.0,False,2.0,False,2.0,False,2.0,False,2.0,0,D,dev,3.928363,[0. 1. 1. 0. 0. 0. 0. 0.],1,"[1, 0, 0, 0, 1, 0, 0, 0]",wrong


In [11]:
df[(df['Sadness']==True) & (df['error']=='wrong')]

,TweetId,tweet,username,mentname,Date,Predom,Feeling,Behavior,Admiration,Admiration_Int,Anger,Anger_Int,Disgust,Disgust_Int,Fear,Fear_Int,Interest,Interest_Int,Joy,Joy_Int,Sadness,Sadness_Int,Surprise,Surprise_Int,group,party,Split,Specificity,emot,group_pred,emot_pred,error
17,1093701723942719489,"John Dingell left a lasting impression with everyone he met. As the longest serving member of the House in history, he will be missed and remembered for his lifetime of service. Farewell to the Dean of the House and deepest condolences to his wife @repdebdingell.",gopleader,repdebdingell,2019-02-07,predom,warm,neutral,False,2.000000,False,2.0,False,2.0,False,2.0,False,2.0,False,2.0,True,2.333333,False,2.0,0,R,dev,4.318822,[0. 0. 0. 0. 0. 0. 1. 0.],1,"[1, 0, 0, 0, 0, 0, 0, 0]",wrong
48,1211705328523522049,"Wishing my friend @repjohnlewis, a man who has more fight in him than anyone I know in Congress, continued strength and courage in his battle. A true American hero.",repjohnyarmuth,repjohnlewis,2019-12-30,predom,warm,app,True,2.666667,False,2.0,False,2.0,False,2.0,False,2.0,False,2.0,True,2.333333,False,2.0,1,D,dev,3.584385,[1. 0. 0. 0. 0. 0. 1. 0.],0,"[1, 0, 0, 0, 0, 0, 0, 0]",wrong
67,1284504286811099136,.@repjohnlewis was a leader who challenged the status quo and helped change our nation for the better... He was a principled gentleman and a friend. He will be missed. My full statement:,senatorwicker,repjohnlewis,2020-07-18,predom,warm,app,True,2.333333,False,2.0,False,2.0,False,2.0,False,2.0,False,2.0,True,2.333333,False,2.0,0,R,dev,3.526774,[1. 0. 0. 0. 0. 0. 1. 0.],1,"[1, 0, 0, 0, 0, 0, 0, 0]",wrong
301,974709958280347649,Extremely sad to hear of the loss of my colleague & dean of the NY delegation @louiseslaughter. She recently brought together the NY delegation in her office. Louise was a staunch advocate for her beliefs and her legacy will forever grace the halls of Congress.,repleezeldin,louiseslaughter,2018-03-16,predom,warm,app,True,2.666667,False,2.0,False,2.0,False,2.0,False,2.0,False,2.0,True,2.333333,False,2.0,0,R,dev,4.460001,[1. 0. 0. 0. 0. 0. 1. 0.],1,"[1, 0, 0, 0, 0, 0, 0, 0]",wrong


In [13]:
df[(df['Admiration']==True) & (df['error']=='wrong')]

,TweetId,tweet,username,mentname,Date,Predom,Feeling,Behavior,Admiration,Admiration_Int,Anger,Anger_Int,Disgust,Disgust_Int,Fear,Fear_Int,Interest,Interest_Int,Joy,Joy_Int,Sadness,Sadness_Int,Surprise,Surprise_Int,group,party,Split,Specificity,emot,group_pred,emot_pred,error
26,1113205272280014848,"Thank you, .@senschumer for thinking of including the Northern Marianas in your amendment to the disaster recovery appropriation bill. Our sincere gratitude for your help.",kilili_sablan,senschumer,2019-04-02,predom,warm,app,True,2.333333,False,2.0,False,2.0,False,2.0,False,2.000000,True,2.000000,False,2.000000,False,2.0,1,D,dev,3.851780,[1. 0. 0. 0. 0. 1. 0. 0.],0,"[1, 0, 0, 0, 0, 0, 0, 0]",wrong
29,1131347509006094336,Parents and families live in constant fear for their children with food allergies. A worthy bipartisan cause - thank you @drphilroe for your leadership on this issue. Thank you Dr. Wood for being a pioneering voice.,reprokhanna,drphilroe,2019-05-22,predom,warm,app,True,2.666667,False,2.0,False,2.0,False,2.0,False,2.333333,True,2.333333,False,2.000000,False,2.0,0,D,dev,4.132768,[1. 0. 0. 0. 0. 1. 0. 0.],1,"[1, 0, 0, 0, 0, 0, 0, 0]",wrong
37,1166016703085469696,"If your house is on fire, you don't have to look up your firehouse's phone number—we have 9-1-1. We should treat mental health emergencies the same way. That's why @repchrisstewart & I want to make 9-8-8 a national mental health emergency line.",teammoulton,repchrisstewart,2019-08-26,predom,warm,app,True,1.666667,False,2.0,False,2.0,False,2.0,False,1.666667,False,1.666667,False,2.000000,False,2.0,0,D,dev,3.576489,[1. 0. 0. 0. 0. 0. 0. 0.],1,"[0, 0, 0, 0, 0, 0, 0, 0]",wrong
40,1186485376296337409,"I’ve never been more proud to serve alongside Chairman @repadamschiff - someone who is fighting for the fate of our democracy and motivated by the impulse to do what’s right, just, and necessary for our country.",repspeier,repadamschiff,2019-10-21,predom,warm,app,True,2.666667,False,2.0,False,2.0,False,2.0,False,2.000000,True,2.000000,False,2.000000,False,2.0,1,D,dev,3.866862,[1. 0. 0. 0. 0. 1. 0. 0.],0,"[1, 0, 0, 0, 0, 1, 0, 0]",wrong
48,1211705328523522049,"Wishing my friend @repjohnlewis, a man who has more fight in him than anyone I know in Congress, continued strength and courage in his battle. A true American hero.",repjohnyarmuth,repjohnlewis,2019-12-30,predom,warm,app,True,2.666667,False,2.0,False,2.0,False,2.0,False,2.000000,False,2.000000,True,2.333333,False,2.0,1,D,dev,3.584385,[1. 0. 0. 0. 0. 0. 1. 0.],0,"[1, 0, 0, 0, 0, 0, 0, 0]",wrong
49,1212171196483211266,Congrats @repcharliecrist on the big win to protect seniors from annoying and fraudulent #robocalls #ForThePeople! Honored to work with you on your great bill!!!,repdarrensoto,repcharliecrist,2019-12-31,predom,warm,app,True,2.666667,False,2.0,False,2.0,False,2.0,False,2.000000,True,2.666667,False,2.000000,False,2.0,1,D,dev,3.553932,[1. 0. 0. 0. 0. 1. 0. 0.],0,"[1, 0, 0, 0, 0, 1, 0, 0]",wrong
67,1284504286811099136,.@repjohnlewis was a leader who challenged the status quo and helped change our nation for the better... He was a principled gentleman and a friend. He will be missed. My full statement:,senatorwicker,repjohnlewis,2020-07-18,predom,warm,app,True,2.333333,False,2.0,False,2.0,False,2.0,False,2.000000,False,2.000000,True,2.333333,False,2.0,0,R,dev,3.526774,[1. 0. 0. 0. 0. 0. 1. 0.],1,"[1, 0, 0, 0, 0, 0, 0, 0]",wrong
69,1288175152656719872,"Small businesses across the country still need our help to get through this rough patch. @senkevincramer’s bipartisan legislation facilitates the forgiveness process, providing increased relief through the #PPP to allow businesses to ensure their employees can earn a paycheck.",senbillcassidy,senkevincramer,2020-07-28,predom,warm,app,True,2.333333,False,2.0,False,2.0,False,2.0,True,2.333333,False,2.000000,False,2.000000,False,2.0,1,R,dev,4.254308,[1. 0. 0. 0. 1. 0. 0. 0.],0,"[0, 0, 0, 0, 1, 0, 0, 0]",wrong
134,431483640950689792,.@sengillibrand thank you for y

In [16]:
df

,TweetId,tweet,username,mentname,Date,Predom,Feeling,Behavior,Admiration,Admiration_Int,Anger,Anger_Int,Disgust,Disgust_Int,Fear,Fear_Int,Interest,Interest_Int,Joy,Joy_Int,Sadness,Sadness_Int,Surprise,Surprise_Int,group,party,Split,Specificity,emot,group_pred,emot_pred,error
0,1005129335160918018,"BREAKING —> The U.S. House just passed a measure I led w\/@repbrianmast to help disabled veterans transition to the civilian workforce by increasing support for the Wounded Warrior Program, which offers these vets 2-yr paid fellowships in House offices. #FlaPol #Sayfie #SSNalerts",repstephmurphy,repbrianmast,2018-06-08,predom,warm,app,False,2.000000,False,2.000000,False,2.000000,False,2.000000,False,2.000000,False,2.000000,False,2.000000,False,2.000000,0,D,dev,4.402458,[0. 0. 0. 0. 0. 0. 0. 0.],0,"[0, 0, 0, 0, 1, 0, 0, 0]",right
1,1007345095153250304,"Today, I introduced the Safe Home Act w\/ @reptommarino to make it clear that child welfare authorities can investigate cases of “rehoming”.",jimlangevin,reptommarino,2018-06-14,predom,warm,app,False,2.000000,False,2.000000,False,2.000000,False,2.000000,False,2.000000,False,1.666667,False,2.000000,False,2.000000,0,D,dev,3.640366,[0. 0. 0. 0. 0. 0. 0. 0.],0,"[0, 0, 0, 0, 0, 0, 0, 0]",right
2,1016423523756204032,"With far too many parents struggling with addiction, grandparents are increasingly coming to the rescue & assuming a parental role. @senbobcasey & I are delighted to announce that our bill to assist these “grandfamilies” has been signed into law.",senatorcollins,senbobcasey,2018-07-09,predom,warm,app,False,2.000000,False,2.000000,False,2.000000,False,2.000000,False,2.000000,True,2.000000,False,2.000000,False,2.000000,0,R,dev,3.960964,[0. 0. 0. 0. 0. 1. 0. 0.],0,"[0, 0, 0, 0, 1, 1, 0, 0]",right
3,1030191720879804416,"With the 2020 Census fast approaching, I joined @SenKamalaHarris and @senatorcarper in introducing legislation to ensure the approximately 10 million Americans who identify as LGBTQ are properly counted and represented in Census data collection efforts.",senduckworth,senatorcarper,2018-08-16,predom,warm,app,False,2.000000,False,2.000000,False,2.000000,False,2.000000,False,2.000000,False,1.666667,False,2.000000,False,2.000000,1,D,dev,4.325275,[0. 0. 0. 0. 0. 0. 0. 0.],1,"[0, 0, 0, 0, 0, 0, 0, 0]",right
4,1033514401473519616,"Rest In Peace, @senjohnmccain. Thank you for serving our country with dignity and honor, as a Navy Pilot, a POW, elected official and a presidential candidate. You were a patriot and a true American hero. My deepest condolences and prayers to his family and loved ones.",repkihuen,senjohnmccain,2018-08-25,predom,warm,app,True,3.000000,False,2.000000,False,2.000000,False,2.000000,False,2.000000,False,2.000000,True,2.333333,False,2.000000,0,D,dev,4.142287,[1. 0. 0. 0. 0. 0. 1. 0.],0,"[1, 0, 0, 0, 0, 0, 0, 0]",right
5,1040280930936672257,"Contrary to what may dominate the headlines, #bipartisanship isn’t dead. My work with @congressmangt (R-PA) to successfully update and expand #CTE across the country proves bipartisan collaboration is still possible in Congress.",jimlangevin,congressmangt,2018-09-13,predom,warm,app,False,2.333333,False,2.000000,False,2.000000,False,2.000000,False,2.000000,False,1.666667,False,2.000000,False,2.000000,0,D,dev,4.139424,[0. 0. 0. 0. 0. 0. 0. 0.],0,"[0, 0, 0, 0, 0, 0, 0, 0]",right
6,1047130609955557376,"Proud to join @repstevestivers to secure $700,000 federal grant to make Ohio’s schools safer places of learning. Get more info now ⬇️⬇️",repbeatty,repstevestivers,2018-10-02,predom,warm,app,False,2.000000,False,2.000000,False,2.000000,False,2.000000,False,2.000000,True,2.333333,False,2.000000,False,2.000000,0,D,dev,3.568885,[0. 0. 0. 0. 0. 1. 0. 0.],1,"[0, 0, 0, 0, 0, 1, 0, 0]",wrong
7,1062044997451579392,"Wishing the happiest of birthdays to my good friend, @senjackreed!",jimlangevin,senjackreed,2018-11-12,predom,warm,neutral,False,2.000000,False,2.000000,False,2.000000,False,2.000000,False,2.000000,True,2.666667,False,2.0

## Getting the dataframe with all mention tweets and making it bigger

In [2]:
mdf = pd.read_csv('../../../../lib-data/congressional-tweets-main/all-mention-tweets.tsv', sep='\t', engine='python').loc[:, ['id', 'tweet', 'username', 'mentname', 'group', 'party']]
mdf.dropna(inplace=True)
mdf['group'] = mdf['group'].astype(int)
mdf

,id,tweet,username,mentname,group,party
0,238056105207554048,"""...to fight for their state like this."" @Randy_Forbes #AR3 #Arkansas",rep_stevewomack,randy_forbes,1,R
1,227768354516131841,Hearing on #MEA is beginning now in House Judiciary Committee #AR3 #Arkansas @RepSpeier,rep_stevewomack,repspeier,-1,R
2,225031228871557120,Thanks 2 everyone who participated in tonight's special Tele-Town Hall w/ @RepTomPrice as we discussed the #healthcare bill. #AR3 #Arkansas,rep_stevewomack,reptomprice,1,R
3,224921430259810306,Tele-Town Hall 2Nite 7PM CT w/ guest @RepTomPrice - Call-in # (877) 229-8493 &amp; passcode 17688 #AR3 #Arkansas,rep_stevewomack,reptomprice,1,R
4,165124042255380481,I am honored to be appointed by @SpeakerBoehner to the US #Military Academy Board of Visitors #AR03,rep_stevewomack,speakerboehner,1,R
...,...,...,...,...,...,...
73180,1387606571824336896,I know I'm not the only one who was moved by @SenatorTimScott's depiction of all the promise &amp; opportunity here in America. We both lived out our American Dream - and we want all the kids across this great nation to find theirs too.,repkevinhern,senatortimscott,1,R
73181,1383077783954460686,Glad to hear it! I’d love to work w/ you on @RepJasonSmith’s HR 1712 to repeal the estate tax (aka the death tax). It’s not fair &amp; it’s not just. The last thing a grieving son or daughter needs is the burden of heavy taxes on their parent’s assets - that have already been taxed!,repkevinhern,repjasonsmith,1,R
73182,1359936343116288002,"Today, Democrats rejected two amendments from @RepArrington &amp; myself, choosing to spend your taxpayer dollars on subsidized healthcare for people in our country illegally while voting against relief for the American energy workers who are unemployed because of Biden’s policies.",repkevinhern,reparrington,1,R
73183,1345829663076179968,"Pelosi’s lack of leadership cost millions of Americans their jobs this year. We’ll be recovering from the damage she caused for years. We need a common-sense leader in the House, which is why I voted for @GOPLeader just now.",repkevinhern,gopleader,1,R


In [3]:
Counter(mdf['group'])

Counter({1: 51105, -1: 22076})

In [4]:
two_df = pd.concat([mdf[mdf['group']==1].sample(3000), mdf[mdf['group']==-1].sample(3000)]).reset_index(drop=True)

three_df = pd.concat([mdf[mdf['group']==1].sample(4500), mdf[mdf['group']==-1].sample(4500)]).reset_index(drop=True)

In [5]:
def train_valid_test():
    t = random.random()
    if t < 0.8:
        return 'train'
    else:
        h = random.random()
        if h < 0.5:
            return 'dev'
        else:
            return 'test'

two_df['Split'] = two_df.apply(lambda x: train_valid_test(), axis=1)
three_df['Split'] = three_df.apply(lambda x: train_valid_test(), axis=1)

two_df.to_csv('two_data.tsv', sep='\t', index=False)
three_df.to_csv('three_data.tsv', sep='\t', index=False)